### in this we will tune the best model that CNN_TF for best parameters

In [ ]:
# IMPORT LIBRARIES
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import ParameterGrid

In [ ]:
# Ensure Eager Execution is Enabled
tf.config.run_functions_eagerly(True)

# DEFINE PATHS
best_model_path = "../3MODEL_SELECTION/MODELS/Best_Model/CNN_TF.keras"
tuned_model_dir = "../5ADVANCED_TRAINING/Tuned_Model"
tuned_model_path = os.path.join(tuned_model_dir, "CNN_TF_Tuned.keras")

# Ensure output directory exists
os.makedirs(tuned_model_dir, exist_ok=True)

# LOAD DATASET
data_path = "../2PREPROCESSING/Processed_CSV"
csv_path = os.path.join(data_path, "cropped_train.csv")

# Load data and check shape
train_data = np.loadtxt(csv_path, delimiter=",", skiprows=1)
num_samples, num_features = train_data.shape
print(f"Loaded dataset shape: {train_data.shape} (Samples: {num_samples}, Features: {num_features})")

# Ensure expected shape (first column = labels, rest = pixels)
expected_feature_size = 20 * 20  # Cropped MNIST is 20x20
assert num_features - 1 == expected_feature_size, f"Unexpected feature size. Expected {expected_feature_size}, got {num_features - 1}"

# Split features (X) and labels (y)
X_train = train_data[:, 1:].reshape(-1, 20, 20, 1)  # Reshape for CNN
y_train = train_data[:, 0].astype(int)

# Normalize pixel values to range [0,1]
X_train = X_train / 255.0

print(f"Reshaped X_train: {X_train.shape}, y_train: {y_train.shape}")

Loaded dataset shape: (60000, 401) (Samples: 60000, Features: 401)
Reshaped X_train: (60000, 20, 20, 1), y_train: (60000,)


In [ ]:
# DEFINE MODEL FUNCTION
def build_model(lr, filters, kernel, dropout, dense_units, activation, pool_size, optimizer):
    if activation == "leaky_relu":
        activation_layer = layers.LeakyReLU()
    else:
        activation_layer = activation  # Keras expects a string activation

    model = models.Sequential([
        layers.Conv2D(filters, (kernel, kernel), activation=activation_layer, input_shape=(20, 20, 1)),
        layers.MaxPooling2D(pool_size),  # Pooling size from parameters
        layers.Conv2D(filters * 2, (kernel, kernel), activation=activation_layer),
        layers.MaxPooling2D(pool_size),
        layers.Flatten(),
        layers.Dense(dense_units, activation=activation_layer),
        layers.Dropout(dropout),
        layers.Dense(10, activation='softmax')  # Output layer (10 classes)
    ])
    
    # Initialize optimizer here instead of inside param_grid
    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

### Parameters To be Used

In [ ]:
# GRID SEARCH: HYPERPARAMETER TUNING
param_grid = {
    'lr': [0.001, 0.003, 0.005],       # Learning Rate
    'filters': [32, 64],               # Number of Conv2D filters
    'kernel': [3, 5],                  # Kernel size
    'dropout': [0.3, 0.5],             # Dropout rate
    'dense_units': [128, 256],         # Number of neurons in Dense layer
    'activation': ['relu', 'leaky_relu'], # Activation function
    'pool_size': [(2, 2)],             # MaxPooling size (fixed from CNN_TF)
    'optimizer': ['adam']  # String instead of object, to initialize later
}

training_params = {
    'batch_size': [128],   # Batch size (fixed from CNN_TF)
    'epochs': [5]          # Number of epochs (fixed from CNN_TF)
}

grid = list(ParameterGrid(param_grid))
best_acc = 0
best_params = None
best_model = None
best_history = None  # To store training history

print(f"Total Hyperparameter Combinations: {len(grid)}")

for i, params in enumerate(grid):
    print(f"\nTesting {i+1}/{len(grid)}: {params}")

    # Build Model (without batch_size & epochs)
    model = build_model(**params)

    # Train Model (using batch_size & epochs separately)
    history = model.fit(
        X_train, y_train, 
        epochs=training_params['epochs'][0],  # Use single fixed value
        batch_size=training_params['batch_size'][0],  # Use single fixed value
        validation_split=0.2, 
        verbose=0
    )

    # Get Validation Accuracy
    val_acc = max(history.history['val_accuracy'])

    # Save Best Model & History
    if val_acc > best_acc:
        best_acc = val_acc
        best_params = params
        best_model = model
        best_history = history.history  # Save the best training history

print(f"\nBest Hyperparameters Found: {best_params}, Accuracy: {best_acc:.4f}")

Total Hyperparameter Combinations: 96

Testing 1/96: {'activation': 'relu', 'dense_units': 128, 'dropout': 0.3, 'filters': 32, 'kernel': 3, 'lr': 0.001, 'optimizer': 'adam', 'pool_size': (2, 2)}


c:\Users\RONIT\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(



Testing 2/96: {'activation': 'relu', 'dense_units': 128, 'dropout': 0.3, 'filters': 32, 'kernel': 3, 'lr': 0.003, 'optimizer': 'adam', 'pool_size': (2, 2)}

Testing 3/96: {'activation': 'relu', 'dense_units': 128, 'dropout': 0.3, 'filters': 32, 'kernel': 3, 'lr': 0.005, 'optimizer': 'adam', 'pool_size': (2, 2)}

Testing 4/96: {'activation': 'relu', 'dense_units': 128, 'dropout': 0.3, 'filters': 32, 'kernel': 5, 'lr': 0.001, 'optimizer': 'adam', 'pool_size': (2, 2)}

Testing 5/96: {'activation': 'relu', 'dense_units': 128, 'dropout': 0.3, 'filters': 32, 'kernel': 5, 'lr': 0.003, 'optimizer': 'adam', 'pool_size': (2, 2)}

Testing 6/96: {'activation': 'relu', 'dense_units': 128, 'dropout': 0.3, 'filters': 32, 'kernel': 5, 'lr': 0.005, 'optimizer': 'adam', 'pool_size': (2, 2)}

Testing 7/96: {'activation': 'relu', 'dense_units': 128, 'dropout': 0.3, 'filters': 64, 'kernel': 3, 'lr': 0.001, 'optimizer': 'adam', 'pool_size': (2, 2)}

Testing 8/96: {'activation': 'relu', 'dense_units': 128, 

In [ ]:
# SAVE BEST MODEL & TRAINING HISTORY
if best_model:
    best_model.save(tuned_model_path)
    print(f"Best tuned model saved at: {tuned_model_path}")
    print(f"Best Model Accuracy: {best_acc:.4f}")

    # Save training history for Step 6 visualization
    history_path = tuned_model_path.replace(".keras", "_history.npy")  
    np.save(history_path, best_history)
    print(f"Training history saved at: {history_path}")

else:
    print("No model was trained.")

Best tuned model saved at: ../5ADVANCED_TRAINING/Tuned_Model\CNN_TF_Tuned.keras
Best Model Accuracy: 0.9844
Training history saved at: ../5ADVANCED_TRAINING/Tuned_Model\CNN_TF_Tuned_history.npy
